In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler

In [3]:
# readfile
train =pd.read_csv('../../raw/CarsSaleForecast/[new] yancheng_train_20171226.csv')
test = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

In [4]:
# duplicate removal
labels = ['sale_date', 'class_id', 'brand_id', 'compartment', 'type_id', 'level_id', 'department_id', 'TR', 'gearbox_type', 'displacement', 'if_charging', 'price_level', 'price', 'driven_type_id', 'fuel_type_id', 'newenergy_type_id', 'emission_standards_id', 'if_MPV_id', 'if_luxurious_id', 'power', 'cylinder_number', 'engine_torque', 'car_length', 'car_width', 'car_height', 'total_quality', 'equipment_quality', 'rated_passenger', 'wheelbase', 'front_track', 'rear_track']
train = train.groupby(labels).sum().reset_index()

In [5]:
# exceptions
train['power'][train['power']=='81/70'] = 81
train['power'] = train['power'].astype('float32')

train['engine_torque'][train['engine_torque']=='155/140'] = 155
train['engine_torque'][train['engine_torque']=='-'] = 201.8
train['engine_torque'] = train['engine_torque'].astype('float32')

train['fuel_type_id'][train['fuel_type_id']==1] = '1'
train['fuel_type_id'][train['fuel_type_id']==2] = '2'
train['fuel_type_id'][train['fuel_type_id']==3] = '3'

train[train['displacement'] == 0] = 2.

In [6]:
# feautres
classLabels = ['class_id']
trainClass = train[classLabels].drop_duplicates().reset_index(drop=True)

In [7]:
# get claaifer features
def get_classifer_features(trainClass, labels):
    for label in labels:
        df = train[['class_id', label]].drop_duplicates()
        dummies = pd.get_dummies(df[label], prefix=label)
        df = df.drop([label], axis=1)
        df = pd.concat([df, dummies], axis=1)
        df = df.groupby('class_id').sum().reset_index()
        trainClass = pd.merge(trainClass, df, on='class_id')
    return trainClass

In [8]:
def get_numerical_features(trainClass, num_labels):
    for label in num_labels:
        df = train[['class_id', label]].drop_duplicates().reset_index(drop=True)
        df[label] = MinMaxScaler().fit_transform(df[label].reshape(-1, 1))
        df_max = df.groupby(df['class_id'])[[label]].agg('max').reset_index()
        df_max.rename(columns={label: label + '_max'},inplace=True)
        df_min = df.groupby(df['class_id'])[[label]].agg('min').reset_index()
        df_min.rename(columns={label:label + '_min'},inplace=True)
        df_mean = df.groupby(df['class_id'])[[label]].agg('mean').reset_index()
        df_mean.rename(columns={label:label + '_mean'},inplace=True)
        df = df.drop([label],axis=1)
        df = df.drop_duplicates().reset_index(drop=True)
        df = pd.merge(df, df_max, on='class_id')
        df = pd.merge(df, df_min, on='class_id')
        df = pd.merge(df, df_mean, on='class_id')
        trainClass = pd.merge(trainClass, df, on='class_id')
    return trainClass

In [9]:
# duplicate removal
labels = ['sale_date', 'class_id', 'brand_id', 'compartment', 'type_id', 'level_id', 'department_id', 'TR', 'gearbox_type', 'displacement', 'if_charging', 'price_level', 'price', 'driven_type_id', 'fuel_type_id', 'newenergy_type_id', 'emission_standards_id', 'if_MPV_id', 'if_luxurious_id', 'power', 'cylinder_number', 'engine_torque', 'car_length', 'car_width', 'car_height', 'total_quality', 'equipment_quality', 'rated_passenger', 'wheelbase', 'front_track', 'rear_track']
train = train.groupby(labels).sum().reset_index()

In [10]:
labels = ['brand_id', 'compartment', 'type_id', 'level_id', 'department_id', 'TR', 'gearbox_type', 'if_charging', 'price_level', 'driven_type_id', 'fuel_type_id', 'newenergy_type_id', 'emission_standards_id', 'if_MPV_id', 'if_luxurious_id', 'cylinder_number', 'rated_passenger']
num_labels = ['displacement', 'power','engine_torque', 'total_quality', 'car_length', 'car_width', 'car_height', 'equipment_quality', 'wheelbase', 'front_track', 'rear_track']

In [11]:
trainClass = get_classifer_features(trainClass, labels)
trainClass = get_numerical_features(trainClass, num_labels)

In [13]:
trainClass.to_csv('../../raw/LiChuan/train_feature.csv', index=False)